In [1]:
import torch as T
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tensorflow.contrib.learn.python.learn.datasets.mnist import extract_images, extract_labels

In [2]:
class AI(nn.Module):
    def __init__(self,conv_kernel):
        super(AI, self).__init__()
        self.conv_kernel = conv_kernel
        self.lstm_layers = 1
        self.lstm_depth = 1000
        self.cnn1 = nn.Conv2d(1,self.conv_kernel,5,stride=2,padding=1)
        self.cnn2 = nn.Conv2d(self.conv_kernel,self.conv_kernel,3,stride=2,padding=0)
        self.rnn = nn.LSTM(6 * 6, self.lstm_depth,self.lstm_layers)
        self.fc1 = nn.Linear(self.lstm_depth,self.lstm_depth)
        self.fc2 = nn.Linear(self.lstm_depth,10)
        self.out = nn.Dropout(0.9)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')     
        self.to(self.device)
    def forward(self,x):
        hx = T.zeros(self.lstm_layers,self.conv_kernel, self.lstm_depth).to(self.device)
        cx = T.zeros(self.lstm_layers,self.conv_kernel, self.lstm_depth).to(self.device)
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn2(x))
#        print(x.shape)
        x = x.view(-1,self.conv_kernel,6 * 6)
        hx, cx = self.rnn(x, (hx, cx))
        o = T.tanh(self.fc1(hx))
        o = T.sigmoid(self.fc2(o))
#        o = self.out(o)
        return o

In [3]:
class PLAYFIELD():
    def __init__(self,width,height):
        self.width = width
        self.height = height
        self.pf = np.zeros((self.width,self.height))
    def checkWin():
        for y in range(self.height):
            count = 0
            for x in range(self.width):
                if self.pf[y][x] == 1:
                    count = count + 1
                else:
                    count = 0
                if count == 3:
                    return 1
        for x in range(self.height):
            count = 0
            for y in range(self.width):
                if self.pf[y][x] == 1:
                    count = count + 1
                else:
                    count = 0
                if count == 3:
                    return 1
        for x in range(self.height):
            count = 0
            for y in range(self.width):
                if self.pf[y][x] == 1:
                    count = count + 1
                else:
                    count = 0
                if count == 3:
                    return 1
    def setStone(self,x,y,stone):
        if self.pf[y][x] == 0:
            self.pf[y][x] = stone
            return 0
        else:
            self.pf[y][x] = -1
            return -1
    def get(self):
        return self.pf

In [4]:
game = PLAYFIELD(3,3)

In [5]:
ai_conv_kernel = 128
ai = AI(ai_conv_kernel)

max_mem = 1000
    
criterion = nn.MSELoss()#SmoothL1Loss()MSELoss
optimizer = optim.RMSprop(ai.parameters(), lr=0.00005)

losses = 0
mul = np.ones([1,28,10])
mul = T.from_numpy(mul).to(ai.device)
for j in range(1):
    for i in range(60000):#train_images.shape[0]):
        def closure():
            optimizer.zero_grad()
            inp1 = train_images[i]#inp[i % max_mem].copy() + np.random.randn(1, a, 28) * 0.1
            inp1.shape = (1,1,28,28)
            inp1 = T.from_numpy(inp1).float().to(ai.device)
            out = ai(inp1)
    #        print(out)
    #        print(target)
            target = out.cpu().detach().numpy().copy()
#            print(target.shape)
#            target[0][8 - 1] = np.zeros(10)
#            target[0][8 - 1][train_labels[i]] = 1.0

#            target[0] = np.zeros([1,8,10])
            n = np.zeros(10)
            n[train_labels[i]] = 1.0
            for o in range(ai_conv_kernel):
                oo = (o + 1) / ai_conv_kernel
                oo = oo * oo
                target[0][o] = oo * n + (1.0 - oo) * target[0][o]
#            target[0][8 - 1][train_labels[i]] = 1.0
#            print(target)
            target = T.from_numpy(np.array(target)).to(ai.device)
            loss = criterion(out, target.float())
            loss.backward()
            return loss
        loss = optimizer.step(closure)
        losses = losses + loss.item()
        if (i % 100) == 0:
            print('Step: ' + str(i) + ' loss: ' + str(losses / 100))
            losses = 0

NameError: name 'train_images' is not defined

In [ ]:
count = 0
for i in range(10000):#train_images.shape[0]):
    optimizer.zero_grad()
    inp1 = test_images[i]#inp[i % max_mem].copy()
    inp1.shape = (1,1,28,28)
    inp1 = T.from_numpy(inp1).float().to(ai.device)
    out = ai(inp1).cpu().detach().numpy()
    m = np.argmax(out[0][5 - 1])
    if m == test_labels[i]:
#        print(m)
        count = count + 1
print("correct: " + str(count / 10000))#train_images.shape[0]))